# Initialization

## Import Libraries

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms


In [2]:
from collections import namedtuple
import tqdm

In [3]:
# make sure GPU is being used 
torch.cuda.current_device() 
torch.cuda.device(0)
torch.cuda.get_device_name(0)

'Tesla V100-SXM2-16GB'

In [4]:
# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
%load_ext autoreload
%autoreload 2

## Import Created Modules

In [5]:
from quantization_functions import quant_aware_layers, quant_aware_resnet_model
from quantization_functions import post_training_quant_layers, post_training_quant_model
from quantization_functions import train_loop

## Load Dataset

In [6]:
BATCH_SIZE = 64
TEST_BATCH_SIZE = 64
N_CLASS = 10

In [7]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])
    
trainset = datasets.CIFAR10(root='./dataCifar', train=True, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)

testset = datasets.CIFAR10(root='./dataCifar', train=False, download=True, transform=transform)

test_loader = torch.utils.data.DataLoader(testset, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# Models

In [8]:
N_EPOCH = 3

## Base model

In [8]:
base_model = torchvision.models.resnet50(pretrained=True)
base_model.fc = nn.Linear(base_model.fc.in_features, N_CLASS) # Change top layer

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/ecbm4040/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
### Train Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(base_model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    epochs=N_EPOCH, save='checkpoint/base_model'
)

train - epoch:  0: : 782it [00:50, 15.59it/s, loss=0.959]                       
val - epoch:  0: : 157it [00:04, 34.02it/s, val_loss=0.61, train_loss=0.959, acc=0.791]                        
train - epoch:  1: : 782it [00:48, 16.08it/s, loss=0.495]                       
val - epoch:  1: : 157it [00:04, 37.57it/s, val_loss=0.533, train_loss=0.495, acc=0.822]                       
train - epoch:  2: : 782it [00:48, 16.00it/s, loss=0.326]                       
val - epoch:  2: : 157it [00:04, 37.16it/s, val_loss=0.533, train_loss=0.326, acc=0.826]                       


In [10]:
base_model.eval()

acc = []
bar = tqdm.tqdm(test_loader)
for x, label in bar:
    x, label = x.cuda(), label.cuda()
    y = base_model(x)
    acc.extend((y.argmax(dim=1) == label).tolist())
    bar.set_postfix({'acc':sum(acc) / len(acc)})
print('acc:', sum(acc) / len(acc))

100%|██████████| 157/157 [00:04<00:00, 37.57it/s, acc=0.826]

acc: 0.8258


## Post Training Quantization

### 8 bit quantization

In [17]:
# Convert base model to a custom quantization layer with the trained weights
c_base_model = quant_aware_resnet_model.CResnet50(num_class=10, q_num_bit=8, pretrained='checkpoint/base_model/model_weights.pt')
c_base_model.quantize(True)

remained state dict odict_keys([])


In [18]:
# Forward pass to have quantized weights
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(c_base_model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=c_base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    epochs=1, save='checkpoint/ptq'
)

train - epoch:  0: : 782it [01:32,  8.41it/s, loss=0.8]                         
val - epoch:  0: : 157it [00:12, 12.41it/s, val_loss=0.733, train_loss=0.8, acc=0.747]                       


In [19]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet50(num_class=10)
q_base_model.convert_from(c_base_model)

In [20]:
q_base_model.eval()

acc = []
bar = tqdm.tqdm(test_loader)
for x, label in bar:
    x, label = x.cuda(), label.cuda()
    y = q_base_model(x)
    acc.extend((y.argmax(dim=1) == label).tolist())
    bar.set_postfix({'acc':sum(acc) / len(acc)})
print('acc:', sum(acc) / len(acc))

100%|██████████| 157/157 [00:09<00:00, 16.15it/s, acc=0.746]

acc: 0.7463


### 4-bit quantization

## Quantization Aware Training

### 8-bit quantization

In [13]:
# Create model with custom quantization layer from the start
c_base_model = quant_aware_resnet_model.CResnet50(num_class=10, q_num_bit=8, pretrained=True)
c_base_model.quantize(True)

remained state dict odict_keys(['fc.weight', 'fc.bias'])


In [14]:
# Training Loop
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(c_base_model.parameters(), 1e-3, momentum=0.9, weight_decay=1e-5)

train_loop.train_model(
    train_dl=train_loader, 
    val_dl=test_loader, 
    model=c_base_model, 
    optimizer=optimizer, 
    criterion=criterion,
    epochs=N_EPOCH, save='checkpoint/qat'
)

train - epoch:  0: : 782it [01:33,  8.39it/s, loss=1.13]                       
val - epoch:  0: : 157it [00:12, 12.81it/s, val_loss=0.885, train_loss=1.13, acc=0.699]                       
train - epoch:  1: : 782it [01:32,  8.46it/s, loss=0.729]                       
val - epoch:  1: : 157it [00:12, 12.40it/s, val_loss=0.723, train_loss=0.729, acc=0.754]                       
train - epoch:  2: : 782it [01:32,  8.41it/s, loss=0.6]                         
val - epoch:  2: : 157it [00:12, 12.46it/s, val_loss=0.676, train_loss=0.6, acc=0.772]                       


In [15]:
# Convert to quantized model
q_base_model = post_training_quant_model.QResnet50(num_class=10)
q_base_model.convert_from(c_base_model)

In [16]:
q_base_model.eval()

acc = []
bar = tqdm.tqdm(test_loader)
for x, label in bar:
    x, label = x.cuda(), label.cuda()
    y = q_base_model(x)
    acc.extend((y.argmax(dim=1) == label).tolist())
    bar.set_postfix({'acc':sum(acc) / len(acc)})
print('acc:', sum(acc) / len(acc))

100%|██████████| 157/157 [00:09<00:00, 16.05it/s, acc=0.772]

acc: 0.7724


### 4-bit quantization